# Example of fitting: 51 Pegasi b

The ELODIE data and parameters used in this example notebook for 51 Peg b  are from [Birkby et al. 2017](http://doi.org/10.3847/1538-3881/aa5c87).

In [ ]:
from ravest.model import Planet, Star, Trend
from ravest.fit import Fitter, Basis
from ravest.param import Parameter
import ravest.prior

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Import the data

In [ ]:
data = pd.read_csv('example_data/51Pegb.txt', delimiter='\s+', )
data

In [ ]:
plt.figure(figsize=(15,3.5))
plt.title("51 Peg b ELODIE data")
plt.ylabel("Radial Velocity [m/s]")
plt.xlabel("BJD_TDB")
plt.errorbar(data["time"], data["vel"], yerr=data["verr"], marker=".", linestyle="None")
plt.show()

In [ ]:
fitter = Fitter(["b"], Basis("per k e w tc"))
fitter.add_data(data["time"], data["vel"], data["verr"])

# Construct the params dict
# These values will be used as your initial guess for the fit
params = {"per_b": Parameter(4.23, "d", fixed=False),
          "k_b": Parameter(60, "m/s", fixed=False),
          "e_b": Parameter(0, "", fixed=True),
          "w_b": Parameter(np.pi/2, "rad", fixed=True),
          "tc_b": Parameter(2456326.9, "d", fixed=False),
          
          "g": Parameter(-33251.9, "m/s", fixed=False),
          "gd": Parameter(0, "m/s/day", fixed=True),
          "gdd": Parameter(0, "m/s/day^2", fixed=True),
          "jit": Parameter(0, "m/s", fixed=True),}

fitter.add_params(params)

fitter.params

In [ ]:
# Construct the priors dict. Every parameter that isn't fixed requires a prior.
priors = {
          "per_b": ravest.prior.Gaussian(4.2293, 0.0011),
          "k_b": ravest.prior.Uniform(0,100),
          "tc_b": ravest.prior.Uniform(2456326.9-(4.2293), 2456326.9+(4.2293)),
          "g": ravest.prior.Uniform(-33260, -33240),
        }

fitter.add_priors(priors)

Now that we have loaded the `Fitter` with the data, our parameters and parameterisation, and defined priors for each of the free parameters, we can now fit the free parameters of the model to the data. First, Maximum A Posteriori (MAP) optimisation is performed to find the best-fit solution. Then, MCMC is used to explore the parameter space and estimate the parameter uncertainties.

In [ ]:
# Fit the free parameters to the data. First performs a
samples = fitter.fit_model_to_data()

Now that the MCMC is finished, the state of the `emcee` sampler has been saved into the `Fitter` object. We can therefore interact with it in the usual way to export the samples, as a a numpy array that can be passed into other functions (such as for comparing two models by calculating the Bayesian evidence - example notebook coming soon!). We can also export them into a Pandas dataframe, which keeps each parameter labelled. In both cases, we can pass in the `discard` and `thin` arguments as desired.

In [ ]:
# Get the results from the sampler
samples = fitter.sampler.get_chain(discard=1000)

# Get the samples as a labelled Pandas dataframe
samples_df = fitter.get_samples_df(discard=1000, thin=1)
samples_df

However, it's far easier to inspect the chains visually. We can plot and optionally save the time series of the parameters in in the chain.

In [ ]:
fitter.plot_chains(discard=1000, save=False)

We can also visualise the posterior parameter distributions using the `corner` module to create corner plots.

In [ ]:
fitter.plot_corner(discard=1000, save=False)